In [15]:
import json
import os

In [17]:
pianos = {}
for dirpath, _, filenames in os.walk("..\\Data\\raw"):
    for filename in filenames:
        if filename.endswith(".json"):
            filepath = os.path.join(dirpath, filename)
            with open(filepath, encoding='utf-8') as json_file:
                data = json.load(json_file)
                if data['piano'] in pianos:
                    pianos[data['piano']] += 1
                else:
                    pianos[data['piano']] = 1
print(pianos)


{'Shigeru Kawai': 3, 'Steinway & Sons': 1}


Pianos statistics